## 🏀 任務說明：將「NBA總冠軍分析器」設計為 Chain-of-Thought（CoT）推理過程

**💡 原始任務：**  
請輸入一支你認為有機會在2024-2025 NBA賽季季後賽中贏得總冠軍的球隊名稱。

---

#### ✅ CoT 改寫版本流程：

1️⃣ **第一階段（思考階段）**  
模擬 AI 分析師的思路，思考並列出「3個該球隊贏得總冠軍的優勢」  
→ 以條列方式呈現，幫助用戶理解該球隊的實力背景。

---

2️⃣ **第二階段（產文階段）**  
從這3個優勢中，挑選出「最能說服其他球隊支持者的亮點」，並用**第一人稱+emoji**風格，撰寫一段簡潔有力的說服分析貼文。

---

📌 **目標：**  
讓用戶透過 AI 協助，獲得具邏輯性又具娛樂感的冠軍預測分析結果。



#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [1]:
import os
from google.colab import userdata

In [2]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# os.environ['MISTRAL']=api_key
# provider = "mistral"
# model = "ministral-8b-latest"

#【使用 OpenAI】
#api_key = userdata.get('OpenAI')
#os.environ['OPENAI_API_KEY']=api_key
#provider = "openai"
#model = "gpt-4o"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [3]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 63.0 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.10.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


### 2. 使用 AISuite 的準備

In [4]:
import aisuite as ai

In [5]:
provider_planner = "groq"
model_planner="llama3-70b-8192"

provider_writer = "groq"
model_writer = "llama3-70b-8192"

#provider_reviewer = "openai"
#model_reviewer = "gpt-4o"

In [6]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 建立二階段思考過程

In [14]:
system_planner = '''請用台灣習慣的中文回應！你的任務是分析並思考一支NBA球隊贏得2024-2025賽季總冠軍的潛在原因。

請根據球隊目前的已知實力、陣容組合特性、戰術風格、核心球星表現趨勢、教練策略與季後賽經驗，思考並列出「3個這支球隊有機會奪得NBA總冠軍的優勢」。

⚠️ 請注意：避免假設資料為最新，若球員狀態、轉隊或教練變動資訊不明確，請使用保守描述（例如「若陣容保持完整」、「假設核心球員健康穩定」）。

請專注於理性分析，避免寫下結論或情緒性語言。
請列出三點，簡潔有力，且具備邏輯性。'''

system_writer = '''請用台灣習慣的中文回應！你的任務是從已經思考好的三個優勢當中，挑選出最具說服力的一點，並以清晰、具邏輯說服力的語氣，將其整理為簡單的列點文字格式。

你的目標是讓一位支持其他球隊的球迷，能夠理解並認同這支球隊奪冠的合理性。

⚠️ 請注意：若相關球員或教練資訊已經有時間落差，請避免使用絕對語氣（例如：「XX球員絕對會帶領球隊奪冠」），可改用條件語句（如：「如果XX球員保持健康，將成為球隊重要優勢」）。

請避免過度誇大，強調邏輯性與事實依據。
最後的輸出請僅包含該球隊名稱 +「最具說服力優勢」的條列式說明，無需額外解釋。'''


In [16]:
def champion_prediction(team_name):
    # Step 1: CoT
    planning_prompt = f"""
使用者輸入的球隊名稱是：{team_name}。

請用台灣習慣的中文回應！請思考這支球隊在2024-2025 NBA賽季能贏得總冠軍的3個優勢，條列清楚。

⚠️ 提醒：由於球員交易與賽季進行期間資訊可能有延遲，請避免假設球員名單或最新狀態，若資訊不明確請使用保守描述（例如：「若陣容維持穩定」或「若主力球員保持健康」）。
"""
    advantages = reply(system_planner, planning_prompt,
                       provider=provider_planner,
                       model=model_planner
                       )

    # Step 2: 從優勢中選出最具說服力的一點，撰寫文章
    generation_prompt = f"""
已經整理好的三個奪冠優勢如下：
{advantages}

請用台灣習慣的中文回應！請從中挑選出最具說服力的一點，並將其轉換為一篇條列式的簡潔說服貼文，讓支持其他球隊的球迷都能認同這支球隊奪冠的合理性。

⚠️ 請注意：若內容涉及球員或教練資訊，請避免使用「已經確定」或「必定會」等絕對語氣，改用「若保持健康」、「若陣容穩定」等條件式描述，確保內容符合時效性與合理性。
"""
    final_post = reply(system_writer, generation_prompt,
                       provider=provider_writer,
                       model=model_writer
                       )

    return advantages, final_post


### 4.  Gradio 建立總冠軍分析器

In [9]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00


In [10]:
import gradio as gr

In [17]:
import gradio as gr

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🏀💡 2024-2025 NBA 總冠軍分析 🌟")
    gr.Markdown("輸入你認為有機會奪冠的球隊名稱，讓 AI 用理性分析 + 說服力整理該隊的奪冠優勢！")

    user_input = gr.Textbox(label="請輸入球隊名稱（如：Boston Celtics）")
    btn = gr.Button("✨ 預測這支球隊為何有機會奪冠！")

    with gr.Row():
        out1 = gr.Textbox(label="🧠 分析結果：3個奪冠優勢（Stage 1 - 分析階段）", lines=5)
        out2 = gr.Textbox(label="🏆 說服貼文：最有力的理由（Stage 2 - 說服階段）", lines=3)

    btn.click(champion_prediction, inputs=[user_input], outputs=[out1, out2])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed50f6a150fc4d7abe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://99e179a0b2d4c602d0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://99e179a0b2d4c602d0.gradio.live
